# Week 5 - a : Generate Lyrics with charRNN
1. 한국 노래 가사 데이터로 훈련셋 구축하기
2. Sequential Dataset을 여러개의 windows로 나누기
3. Building & Training the CharRNN model
4. Play!

In [1]:
# 이번에는 tensorflow를 사용해봅시다!
import tensorflow as tf  # ready-made RNN을 사용하기 위해!
from tensorflow import keras  # tensorflow 안에 있는 keras 라이브러리를 사용!
import numpy as np
import requests
import os
import time

In [2]:
# gpu 사용가능 여부 체크
# 출처: https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=Y04m-jvKRDsJ
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## 1. 훈련셋 구축하기

한국어 가사 데이터셋을 다운로드 하여, RNN학습을 위한 데이터 셋을 구축해봅시다.


In [3]:
# 한국어 발라드 가사 데이터를 불러오기.
LYRICS_URL: str = 'https://raw.githubusercontent.com/ahastudio/ballad-lyrics-maker/master/data/lyrics_ballad/input.txt'
corpus = requests.get(LYRICS_URL).text

In [4]:
# 첫 200글자 데이터 확인.
# corpus: List[str] (x)
# corpus: str
print(type(corpus))
print(corpus[:1000])

<class 'str'>
내 곁에서 떠나가지 말아요 
그대없는 밤은 너무 쓸쓸해 
그대가 더 잘 알고 있잖아요 
제발 아무말도 하지 말아요
나약한 내가 뭘 할수 있을까 생각을 해봐
그대가 내겐 전부였었는데 음~오 
제발 내 곁에서 떠나가지 말아요
그대없는 밤은 너무 싫어
우~우~우~ 돌이킬수 없는 그대 마음 
우~우~우~ 이제와서 다시 어쩌려나
슬픔마음도 이젠 소용없네 

내 곁에서 떠나가지 말아요 
그대없는 밤은 너무 쓸쓸해 
그대가 더 잘 알고 있잖아요 
제발 아무말도 하지 말아요
나약한 내가 뭘 할수 있을까 생각을 해봐
그대가 내겐 전부였었는데 음~오 
제발 내 곁에서 떠나가지 말아요
그대없는 밤은 너무 싫어
우~우~우~ 돌이킬수 없는 그대 마음 
우~우~우~ 이제와서 다시 어쩌려나
슬픔마음도 이젠 


조용한 밤하늘에
아름다운 별빛이 
멀리 있는 창가에도
소리 없이 비추고 
한낮의 기억들은
어디론가 사라져 
꿈을 꾸는 저 하늘만
바라보고 있어요 
부드러운 노래 소리에
내 마음은 아이처럼 
파란 추억의 바다로 
뛰어가고 있네요 
깊은 밤 아름다운 그 시간은 
이렇게 찾아와 마음을 물들이고 
영원한 여름밤의 꿈을
기억하고 있어요 
다시 아침이 밝아와도
잊혀지지 않도록
부드러운 노래 소리에
내 마음은 아이처럼 
파란 추억의 바다로 
뛰어가고 있네요 
깊은 밤 아름다운 그 시간은 
이렇게 찾아와 마음을 물들이고 
영원한 여름밤의 꿈을
기억하고 있어요 
다시 아침이 밝아와도
잊혀지지 않도록

나의 하늘을 본 적이 있을까
조각 구름과 빛나는 별들이
끝없이 펼쳐 있는
구석진 그 하늘 어디선가
내 노래는 널 부르고 있음을
너는 듣고 있는지
나의 정원을 본 적이 있을까
국화와 장미 예쁜 사루비아가
끝없이 피어 있는
언제든 그 문은 열려 있고
그 향기는 널 부르고 있음을
넌 알고 있는지
나의 어릴 적 내 꿈만큼이나
아름다운 가을 하늘이랑
네가 그것들과 손잡고
고요한 달빛으로 내게 오면
내 여린 마음으로 피워낸
나의 사랑을
너에게 꺾어줄게
나의 어릴 적 내 꿈만큼이나
아름

In [5]:
corpus[0] # 첫번쨰 "데이터" = 하나의 글자

'내'

In [6]:
corpus[:10]

'내 곁에서 떠나가지'

In [8]:
# 다음으로는, 먼저 각 글자별 정수 인코딩을 진행해주어야합니다.
# char_level = True로 설정해주면, 단어가 아닌 글자 단위로 정수 인코딩을 해줍니다.
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # 단어로 쪼개준다
# 말뭉치에 토크나이저를 학습시킵니다. 각 글자에 대응하는 정수 인코딩을 찾아줍니다!
tokenizer.fit_on_texts([corpus])

In [9]:
# 한번 예시를 살펴볼까요?
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences
print(tokenizer.texts_to_sequences(["난 너를 사랑해"]))
print(tokenizer.sequences_to_texts([[59, 1, 38, 35, 1, 21, 27, 28]]))

[[59, 1, 38, 35, 1, 21, 27, 28]]
['난   너 를   사 랑 해']


In [ ]:
# tokenizer가 좋은 이유는 여러가지 메타데이터가 있다
# word_index : 어떠한 글자에 대응하는 정수를 담고 있는 딕셔너리. 이것의 크기를 알면 고유한 글자의 개수를 알 수 있다
# word_counts : 말뭉치 속의 고유한 단어들의 빈도 수

In [10]:
# 말뭉치 내에 존재하는 각 고유한 글자의 빈도수를 계산해준다. 
print(tokenizer.word_counts)

OrderedDict([('내', 6213), (' ', 174203), ('곁', 642), ('에', 8619), ('서', 3674), ('떠', 2010), ('나', 11700), ('가', 11345), ('지', 11258), ('말', 2705), ('아', 9481), ('요', 5041), ('\n', 62341), ('그', 12756), ('대', 6299), ('없', 3598), ('는', 11818), ('밤', 1033), ('은', 6232), ('너', 3421), ('무', 2566), ('쓸', 381), ('해', 4015), ('더', 927), ('잘', 337), ('알', 1018), ('고', 6687), ('있', 3642), ('잖', 303), ('제', 1759), ('발', 458), ('도', 6134), ('하', 7536), ('약', 149), ('한', 3875), ('뭘', 62), ('할', 1238), ('수', 2980), ('을', 8483), ('까', 1836), ('생', 1495), ('각', 1200), ('봐', 561), ('겐', 208), ('전', 632), ('부', 1390), ('였', 213), ('었', 1643), ('데', 1252), ('음', 3178), ('~', 669), ('오', 3050), ('싫', 116), ('어', 10315), ('우', 3136), ('돌', 984), ('이', 15932), ('킬', 31), ('마', 3198), ('와', 1225), ('다', 7306), ('시', 2876), ('쩌', 100), ('려', 1975), ('슬', 813), ('픔', 419), ('젠', 762), ('소', 1483), ('용', 137), ('네', 2965), ('조', 559), ('늘', 1802), ('름', 1256), ('운', 1740), ('별', 768), ('빛', 740), ('멀', 895), ('

In [11]:
# 고유한 글자의 개수는?
vocab_size = len(tokenizer.word_index)
print(vocab_size)
# 전체 데이터셋의 크기는?
dataset_size = sum([
                count
                # word_counts: Dict[Tuple[str=글자, int=빈도수]]
                # -> items -> List[Tuple[str, count]]
                # -> loop -> str, int -> int
                for character, count in tokenizer.word_counts.items()
])
print(dataset_size)

1207
742966


In [12]:
# 텍스트 데이터 -> 정수 인코딩 데이터로 전처리하기
[corpus_encoded] = np.array(tokenizer.texts_to_sequences(texts=[corpus]))

In [13]:
# 글자 -> 정수
print(corpus[:100])
print(corpus_encoded[:100])

내 곁에서 떠나가지 말아요 
그대없는 밤은 너무 쓸쓸해 
그대가 더 잘 알고 있잖아요 
제발 아무말도 하지 말아요
나약한 내가 뭘 할수 있을까 생각을 해봐
그대가 내겐 전부였었는데
[ 19   1 158  11  32   1  58   6   7   8   1  49  10  25   1   2   4  17
  34   5   1 115  18   1  38  51   1 232 232  28   1   2   4  17   7   1
 120   1 241   1 116  15   1  33 255  10  25   1   2  67 209   1  10  51
  49  20   1  13   8   1  49  10  25   2   6 374  30   1  19   7   1 514
   1 104  43   1  33  12  63   1  81 107  12   1  28 176   2   4  17   7
   1  19 312   1 161  89 306  73   5 101]


### 학습에 필요한 데이터 구축하기


In [14]:
# 전체 데이터셋의 90%만 훈련셋으로 사용
train_size = dataset_size * 90 // 100
# 리스트 슬라이싱으로 train & test 구분 
# Dataset.from_tensor_slices: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices
ds = tf.data.Dataset.from_tensor_slices(corpus_encoded[:train_size])
# 텐서가 있는 데이터셋으로 만들어 준 것

In [15]:
# ds: Dataset[Tensor]
# 각 sample은 하나의 숫자를 담고 있는 Tensor
# ds.take: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take
for sample in ds.take(3):
  print(sample)

tf.Tensor(19, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(158, shape=(), dtype=int64)


In [16]:
# 앞 70% 노래가사 = 훈련셋, 뒤 30% 노래가사 = 테스트셋.
corpus_encoded

array([ 19,   1, 158, ...,   1,   2,   2])



![image](https://user-images.githubusercontent.com/56193069/131788910-1b620100-d6e2-4e34-b60c-9fe93f09f5dc.png)

과거에 존재하는 패턴은 현재 혹은 미래에도 나타날 것이라고 함부로 가정할 수 없기 때문에,
앞 -뒤로 훈련/테스트 셋을 스플릿하면 과적합이 일어날 수 있다는 문제가 발생.

그래서 시간에 민감한 시계열 데이터의 경우, 구간을 나누어서 훈련-테스트셋을 구축한다.
0.7*[2020-2019] , 0.7*[2019-2018], 0.7*[2017-2018]


## 2. Sequential Dataset을 여러개의 windows로 나누기

In [17]:
WINDOW_SIZE = 100 
# Dataset.window(): https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices
# target = input shifted 1 character ahead. (그래서 shift=1)로 설정함.
ds = ds.window(WINDOW_SIZE, shift=1, drop_remainder=True)

In [18]:
# ds: Dataset[Dataset] 
# 그런데 우리가 원하는 형식 =Dataset[Tensor]
# 완성된 윈도우 확인 - 하나씩 shift되어 있는 것을 볼 수 있음!
for sample in ds.take(3):
  print(list(sample.as_numpy_iterator()))

[19, 1, 158, 11, 32, 1, 58, 6, 7, 8, 1, 49, 10, 25, 1, 2, 4, 17, 34, 5, 1, 115, 18, 1, 38, 51, 1, 232, 232, 28, 1, 2, 4, 17, 7, 1, 120, 1, 241, 1, 116, 15, 1, 33, 255, 10, 25, 1, 2, 67, 209, 1, 10, 51, 49, 20, 1, 13, 8, 1, 49, 10, 25, 2, 6, 374, 30, 1, 19, 7, 1, 514, 1, 104, 43, 1, 33, 12, 63, 1, 81, 107, 12, 1, 28, 176, 2, 4, 17, 7, 1, 19, 312, 1, 161, 89, 306, 73, 5, 101]
[1, 158, 11, 32, 1, 58, 6, 7, 8, 1, 49, 10, 25, 1, 2, 4, 17, 34, 5, 1, 115, 18, 1, 38, 51, 1, 232, 232, 28, 1, 2, 4, 17, 7, 1, 120, 1, 241, 1, 116, 15, 1, 33, 255, 10, 25, 1, 2, 67, 209, 1, 10, 51, 49, 20, 1, 13, 8, 1, 49, 10, 25, 2, 6, 374, 30, 1, 19, 7, 1, 514, 1, 104, 43, 1, 33, 12, 63, 1, 81, 107, 12, 1, 28, 176, 2, 4, 17, 7, 1, 19, 312, 1, 161, 89, 306, 73, 5, 101, 1]
[158, 11, 32, 1, 58, 6, 7, 8, 1, 49, 10, 25, 1, 2, 4, 17, 34, 5, 1, 115, 18, 1, 38, 51, 1, 232, 232, 28, 1, 2, 4, 17, 7, 1, 120, 1, 241, 1, 116, 15, 1, 33, 255, 10, 25, 1, 2, 67, 209, 1, 10, 51, 49, 20, 1, 13, 8, 1, 49, 10, 25, 2, 6, 374, 30, 1, 1

In [19]:
# lambda function = anonymous function
# 함수를 굳이 정의하지 않고 함수의 행위만을 정의하고 싶을 때.
# "처리"를 하는 것만 하고 싶고, 어떻게 리스트를 처리할지는, 사용자에게 맡기고 싶다.

def process(l: list, proc_func) -> list:
  return [
          proc_func(elem)
          for elem in l
  ]



def square(elem: int) -> int:
  return elem**2
  
def add_1(elem: int) -> int:
  return elem + 1
 

# def triple(elem: int) -> int:
#   return elem**3

In [20]:
my_list = [1, 2, 3]
process(my_list, proc_func=square)

[1, 4, 9]

In [21]:
process(my_list, proc_func=add_1)

[2, 3, 4]

In [22]:
# 세제곱을 하고 싶다.
# 굳이 함수를 정의하지 않아도, 원하는 행위만을 정의해서 사용할 수 있다.
# 함수긴 하지만, 이름이 존재하지는 않음 -> "anonymous function"
# https://www.programiz.com/python-programming/anonymous-function
process(my_list, proc_func=lambda elem: elem**3)

[1, 8, 27]

In [23]:
# Dataset 안에 또다른 Dataset이 있다. 
# -> Dataset 속에 Tensor가 있도록 바꿔주자!
ds = ds.flat_map(map_func=lambda window: window.batch(WINDOW_SIZE))

In [24]:
# ds: Dataset[Tensor] 확인하기!
for sample in ds.take(3):
  print(sample)

tf.Tensor(
[ 19   1 158  11  32   1  58   6   7   8   1  49  10  25   1   2   4  17
  34   5   1 115  18   1  38  51   1 232 232  28   1   2   4  17   7   1
 120   1 241   1 116  15   1  33 255  10  25   1   2  67 209   1  10  51
  49  20   1  13   8   1  49  10  25   2   6 374  30   1  19   7   1 514
   1 104  43   1  33  12  63   1  81 107  12   1  28 176   2   4  17   7
   1  19 312   1 161  89 306  73   5 101], shape=(100,), dtype=int64)
tf.Tensor(
[  1 158  11  32   1  58   6   7   8   1  49  10  25   1   2   4  17  34
   5   1 115  18   1  38  51   1 232 232  28   1   2   4  17   7   1 120
   1 241   1 116  15   1  33 255  10  25   1   2  67 209   1  10  51  49
  20   1  13   8   1  49  10  25   2   6 374  30   1  19   7   1 514   1
 104  43   1  33  12  63   1  81 107  12   1  28 176   2   4  17   7   1
  19 312   1 161  89 306  73   5 101   1], shape=(100,), dtype=int64)
tf.Tensor(
[158  11  32   1  58   6   7   8   1  49  10  25   1   2   4  17  34   5
   1 115  18   1  38  51

In [25]:
# 로스를 구하는 단위 = batch
# 로스를 구할 때, 32개의 window를 대상으로 로스를 계산하자.
# 배치 구성.
BATCH_SIZE = 32
ds = ds.shuffle(10000).batch(BATCH_SIZE)

In [26]:
for batch in ds.take(3):
  # 하나의 배치 = 32개의 윈도우가 들어있고, 각 윈도우의 크기는 100
  print(batch.shape)

(32, 100)
(32, 100)
(32, 100)


In [27]:
# x & y 만들기 (입력 & label)
# 입력과 정답을 만드는 과정.
# rnn 입력=나열, 출력=나열
# e.g. 
# data =난, 너, 를,사, 랑, 해
# input = 난,너,를,사,랑
# target =너,를,사,랑,해
# P(next|context)
#즉, P(너|난), P(를|난,너), P(사|난,너,를), P(랑|난,너,를,사), P(해|난,너,를,사,랑)를 최대화 하도록 RNN의 가중치를 학습시키고자 하는 것.
ds = ds.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

In [28]:
for batch in ds.take(1):
  # 하나의 배치 = 32개의 윈도우가 들어있고, 각 윈도우의 크기는 100
  X, Y = batch
  # X (32, 100)
  # Y (32, 100)
  print(X[0])
  print("---")
  print(Y[0])

tf.Tensor(
[  1  90   8   1 118  82  14  31   2  30   1 190 339   1  26   6   1  45
  36  53   2 182 117  24   1   3  55  29  35   1  81 107  82 139   2   9
 281  12   1   7  23   8  88  94   1  19  16   5   1 304  65  12   2  30
 254   1 240  80   1  32 278   3  14   2 108  11   1 354  18   1  19   1
  47 103   3   1 108 362  91   2  54 102   1 295  11  32   1  89 213  34
   5   1  87   3  36   1  49  82 139], shape=(99,), dtype=int64)
---
tf.Tensor(
[ 90   8   1 118  82  14  31   2  30   1 190 339   1  26   6   1  45  36
  53   2 182 117  24   1   3  55  29  35   1  81 107  82 139   2   9 281
  12   1   7  23   8  88  94   1  19  16   5   1 304  65  12   2  30 254
   1 240  80   1  32 278   3  14   2 108  11   1 354  18   1  19   1  47
 103   3   1 108 362  91   2  54 102   1 295  11  32   1  89 213  34   5
   1  87   3  36   1  49  82 139   2], shape=(99,), dtype=int64)


In [29]:
# prefetch: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take
# 더 빠른 훈련을 위해 필요. 하나의 캐릭터를 처리할 때, 미리 다음 캐릭터를 로드해두도록 할 수 있음.
# HDD <-> Memory 로 데이터를 가져오는데 발생하는 병목현상을 완화할수 있어요.
# HDD = 느려요.
# Memory = 빠릅니다. 
# 비유를 하자면 - 매번 도서관에 갔다오는 것보다, 월요일에 읽을책을 전부다 미리 빌려놓고,
# 책상위에서 책을 가져오는 것과 같은 격.
ds = ds.prefetch(1)

## 3. Building & Training the CharRNN model

In [ ]:
# [난,너,를,사]
# [1, 2, 3, 4] 이렇게 모델에 입력으로 넣게되면,
#  난 < 사, 너 < 사.  대소관계를 학습하게 된다.
# [[1000], [0100], [0010], [0001]]
# 그래서 원핫벡터로 전처리를 한다.

In [30]:
EMBEDDING_DIM = 100
RNN_UNITS = 1024
def load_rnn():
  model = tf.keras.Sequential([
    # one-hot encoding 대신, embedding 레이어를 사용!                               
    tf.keras.layers.Embedding(vocab_size, EMBEDDING_DIM,
                              batch_input_shape=[BATCH_SIZE, None]), # None : 어떠한 길이도 허용해준다
    # 간단한 RNN 레이어 하나.                               
    tf.keras.layers.SimpleRNN(RNN_UNITS,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),

    # 마지막은 모든 글자에 대한 점수를 출력하는 linear layer
    tf.keras.layers.Dense(vocab_size)])
  return model

# 분류문제를 푸는 것이므로, cross entropy loss를 사용한다.
def loss(labels, logits):
    # 출력이 logits이므로,  from_logits를 true로 두어야 한다.
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model = load_rnn()
# 분류문제를 푸는 것이므로, cross entropy loss를 사용한다.
model.compile(loss=loss, optimizer="adam", metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 100)           120700    
_________________________________________________________________
simple_rnn (SimpleRNN)       (32, None, 1024)          1152000   
_________________________________________________________________
dense (Dense)                (32, None, 1207)          1237175   
Total params: 2,509,875
Trainable params: 2,509,875
Non-trainable params: 0
_________________________________________________________________


### To-do 2
다음의 질문에 답하세요:

입력값을 원-핫 인코딩을 하게될 경우, 위처럼 굳이 임베딩 레이어를 추가할 필요는 없습니다. 하지만 우리는 원핫 인코딩보다는 임베딩 레이어를 추가하는 것이 더 적절한 상황입니다. 그 이유는 무엇일까요?
( hint: 고유한 글자가 몇개있나요?)

---
답:

고유한 단어의 개수 = 1207 = 클래스의 개수 1207개.
원핫벡터의 차원이 1207. 희소한 벡터를 데이터로 쓰게되는 것.
당연히 학습에 필요한 데이터도 많아지겠죠. 그래서 학습이 필요하더라도
차원을 작게두고 각 글자의 임베딩 벡터를 학습을 하는 것이 나을 것.

- 이정무15:31
원핫백터 처리하기에는 유니크한 글자수가 너무 많다
1207 * 윈도우 갯수
차라리 임배딩 레이어를 통해 특징을 인코딩 하는게 더 효율적이다
---

In [31]:
# 체크포인트가 저장될 디렉토리
# 이걸 현재까지 가장 나은 모델을 알아서 저장해준다.
CHECKPOINT_PATH = './models/my_checkpt.ckpt'
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=CHECKPOINT_PATH,
    save_weights_only=True, 
    save_best_only=True,
    monitor='loss', 
    verbose=1)


In [32]:
# 캐릭터 분류문제이므로, categorical cross entropy loss를 사용 
# GPU를 사용해도, 훈련에 상당한 시간이 걸릴 것.
# 그래서 epoch은 3으로만 설정하겠다!
EPOCHS = 10
STEPS_PER_EPOCH = 172
history = model.fit(ds,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=EPOCHS,
                    callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 29s 138ms/step - loss: 3.8795 - acc: 0.2609

Epoch 00001: loss improved from inf to 3.87952, saving model to ./models/my_checkpt.ckpt
Epoch 2/10
172/172 [==============================] - 24s 138ms/step - loss: 1.8984 - acc: 0.6068

Epoch 00002: loss improved from 3.87952 to 1.89839, saving model to ./models/my_checkpt.ckpt
Epoch 3/10
172/172 [==============================] - 24s 138ms/step - loss: 0.9057 - acc: 0.8349

Epoch 00003: loss improved from 1.89839 to 0.90566, saving model to ./models/my_checkpt.ckpt
Epoch 4/10
172/172 [==============================] - 24s 137ms/step - loss: 0.7178 - acc: 0.8722

Epoch 00004: loss improved from 0.90566 to 0.71777, saving model to ./models/my_checkpt.ckpt
Epoch 5/10
172/172 [==============================] - 24s 137ms/step - loss: 0.6509 - acc: 0.8863

Epoch 00005: loss improved from 0.71777 to 0.65091, saving model to ./models/my_checkpt.ckpt
Epoch 6/10
172/172 [========================

In [ ]:
# 가장 성능이 좋은 모델을 로드
model = load_rnn()
model.load_weights(CHECKPOINT_PATH)
model.build(tf.TensorShape([1, None]))
model.summary()

In [ ]:
def generate_text(model, start_string: str, temp: float, num_chars: int):
    global tokenizer
    # --- temp --- #
    # 온도가 높으면 더 의외의 텍스트가 됩니다.
    # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
    # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)
    # 시작 문자열을 숫자로 변환(벡터화)
    input_eval = tokenizer.texts_to_sequences(start_string)
    input_eval = tf.expand_dims(input_eval, 0)
    # 결과를 저장할 빈 문자열
    text_generated = []
    # 최적의 세팅을 찾기 위한 실험
    # 여기에서 배치 크기 == 1
    model.reset_states()
    for i in range(num_chars):
        predictions = model(input_eval)
        # 배치 차원 제거
        predictions = tf.squeeze(predictions, 0)
        # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
        predictions = predictions / temp
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        # 예측된 단어를 다음 입력으로 모델에 전달
        input_eval = tf.expand_dims([predicted_id], 0)
        decoded = tokenizer.index_word[predicted_id]
        print(decoded)
        text_generated.append(decoded)
    return (start_string + ''.join(text_generated))

## 4. Play!

In [ ]:
print(generate_text(model, start_string=u"그대를 만나고\n", temp=0.8, num_chars=100))

In [ ]:
print(generate_text(model, start_string=u"그리워하면 언젠간 만나게 되는", temp=0.8, num_chars=100))

In [ ]:
print(generate_text(model, start_string=u"또 하루 멀어져", temp=0.8, num_chars=100))

In [ ]:
# 기훈님
print(generate_text(model, start_string="꽃이 예뻐봤자 뭐해\n", temp=0.8, num_chars=100))

In [ ]:
# 경서님
print(generate_text(model, start_string="피노키오\n", temp=0.8, num_chars=100))

In [ ]:
# 경서님
print(generate_text(model, start_string="피노키오 ", temp=0.8, num_chars=100))

In [ ]:
# 선영님
print(generate_text(model, start_string="광야로 걸어가 ", temp=0.8, num_chars=100))

In [ ]:
# 찬우님
print(generate_text(model, start_string="아 뜨거워 아 뜨거워 주님의 사랑\n", temp=0.8, num_chars=100))